<a href="https://colab.research.google.com/github/younggon2/Education-ComputerVision-DeepLearning/blob/master/Day2-1%20Keras%20CNN%20(Classification).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

# Keras 실습 (Cont.)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
%matplotlib inline

In [ ]:
from tensorflow.keras.datasets import fashion_mnist
((trainX, trainY), (testX, testY)) = fashion_mnist.load_data()

# initialize the label name
labelNames = ["top", "trouser", "pullover", "dress", "coat",
              "sandal", "shirt", "sneaker", "bag", "ankle boot"]

In [ ]:
# flatten 28*28 images to a 784 vector for each image
width = height = 28
num_pixels = width * height
trainX = trainX.reshape(60000, num_pixels).astype('float32') / 255.0
testX = testX.reshape(10000, num_pixels).astype('float32') / 255.0

# 훈련셋과 검증셋 분리
valX = trainX[50000:]
valY = trainY[50000:]
trainX = trainX[:50000]
trainY = trainY[:50000]

# one hot encode outputs
num_classes = 10
trainY = tf.keras.utils.to_categorical(trainY, num_classes)
valY = tf.keras.utils.to_categorical(valY, num_classes)
testY = tf.keras.utils.to_categorical(testY, num_classes)

print ('train shape: \t', trainX.shape)
print ('valid shape: \t', valX.shape)
print ('test shape: \t', testX.shape)

## STEP 11: 네 번째 인공지능 모델 (Convolutional Neural Network, CNN)
![대체 텍스트](https://www.mdpi.com/entropy/entropy-19-00242/article_deploy/html/images/entropy-19-00242-g001.png)
**중요! 입력 데이터의 형태가 바뀌어야 한다!!!!**  
**784 (1D) -> 28x28 (2D)**

In [ ]:
# reshape to be [samples][pixels][width][height]
trainX = trainX.reshape(50000, 28, 28, 1)
valX = valX.reshape(10000, 28, 28, 1)
testX = testX.reshape(10000, 28, 28, 1)

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

def simple_cnn_model():
    # create model
    model = Sequential()

    model.add(Conv2D(32, (5,5), input_shape=(28, 28, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    # Compile model
    sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['acc'])

    return model

In [ ]:
# build the model
model = simple_cnn_model()
model.summary()

# fix random seed for reproductibility
seed = 7
np.random.seed(seed)
tf.random.set_seed(seed)

# Fit the model
hist = model.fit(trainX, trainY, validation_data=(valX, valY), epochs=20, batch_size=64, verbose=1)
model.save('simple_cnn_model.h5')

# 5. 학습과정 살펴보기
fig, loss_ax = plt.subplots()

acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
loss_ax.set_ylim([0.0, 1.5])

acc_ax.plot(hist.history['acc'], 'b', label='train acc')
acc_ax.plot(hist.history['val_acc'], 'g', label='val acc')
acc_ax.set_ylim([0.5, 1.0])

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()

In [ ]:
# Final evaluation of the model
scores = model.evaluate(testX, testY, verbose=0)
print("2D simple CNN error: %.2f%%" % (100-scores[1]*100))

## STEP 12: Convolution kernel 살펴보기 (5x5)

In [ ]:
W1 = model.layers[0].get_weights()[0]
W1 = np.squeeze(W1)

print(W1.shape)
W1 = np.transpose(W1, (2,0,1))

plt.figure(figsize=(5, 5), frameon=False)
for ind, val in enumerate(W1):
    plt.subplot(6, 6, ind + 1)
    im = val.reshape((5,5))
    plt.axis("off")
    plt.imshow(im, cmap='gray',interpolation='nearest')

In [ ]:
import tensorflow as tf

convout1_f = tf.keras.models.Model([model.layers[0].input], [model.layers[1].output])

x_rep = convout1_f([testX[0:3]])
x_rep = np.squeeze(x_rep)

print(x_rep.shape)

for this_x_rep in x_rep:
    plt.figure(figsize=(5, 5), frameon=False)

    for i in range (this_x_rep.shape[2]):
        val = this_x_rep[:,:,i]
        plt.subplot(6, 6, i + 1)
        plt.axis("off")
        plt.imshow(val, cmap='gray',interpolation='nearest')

## STEP 13: 마지막 인공지능 모델 (VGG-like CNN)
![대체 텍스트](https://neurohive.io/wp-content/uploads/2018/11/vgg16-1-e1542731207177.png)

In [ ]:
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout

def cnn_model():
    # create model
    model = Sequential()

    model.add(Conv2D(32, (3,3), input_shape=(28, 28, 1)))
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))

    model.add(Conv2D(32, (3,3)))
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))

    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(64, (3,3)))
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))

    model.add(Conv2D(64, (3,3)))
    model.add(BatchNormalization())
    model.add(Activation(activation='relu'))

    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    # Compile model
    sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['acc'])

    return model

In [ ]:
# build the model
model = cnn_model()

# fix random seed for reproductibility
seed = 7
np.random.seed(seed)
tf.random.set_seed(seed)

# Fit the model
hist = model.fit(trainX, trainY, validation_data=(valX, valY), epochs=20, batch_size=64, verbose=1)
model.save('cnn_model.h5')

# 5. 학습과정 살펴보기
fig, loss_ax = plt.subplots()

acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
loss_ax.set_ylim([0.0, 1.5])

acc_ax.plot(hist.history['acc'], 'b', label='train acc')
acc_ax.plot(hist.history['val_acc'], 'g', label='val acc')
acc_ax.set_ylim([0.5, 1.0])

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()

In [ ]:
# Final evaluation of the model
scores = model.evaluate(testX, testY, verbose=0)
print("VGG-like CNN error: %.2f%%" % (100-scores[1]*100))

## STEP 14: 결과 확인하기 (틀린 것 들만)

In [ ]:
# 7. 모델 사용하기
yhat_test = model.predict(testX, batch_size=32)

plt_row = 5
plt_col = 5

plt.rcParams["figure.figsize"] = (20,20)

f, axarr = plt.subplots(plt_row, plt_col)

cnt = 0
i = 0

while cnt < (plt_row*plt_col):

    if np.argmax(testY[i]) == np.argmax(yhat_test[i]):
        i += 1
        continue

    sub_plt = axarr[(int)(cnt/plt_row), cnt%plt_col]
    sub_plt.axis('off')
    sub_plt.imshow(testX[i].reshape(width, height), cmap='gray')
    sub_plt_title = 'R: ' + labelNames[np.argmax(testY[i])] + '(%.2f)'% (yhat_test[i][np.argmax(testY[i])]) + ' P: ' + labelNames[np.argmax(yhat_test[i])] + '(%.2f)'% (  yhat_test[i][np.argmax(yhat_test[i])])
    sub_plt.set_title(sub_plt_title)

    i += 1
    cnt += 1

plt.show()

# Medical Image Classification

## 1. 데이터 준비: MedNIST dataset

In [ ]:
# 데이터 다운로드
!wget https://raw.githubusercontent.com/mi2rl/datasets/master/mednist.tar.gz

In [ ]:
# 압축 풀기
!tar xzf mednist.tar.gz

In [ ]:
import os
import cv2
import time
import random
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
dataDir = 'resized'                                         # 데이터 위치
classNames = sorted(os.listdir(dataDir))                    # 각 클래스의 이름들
numClass = len(classNames)                                  # Number of classes = number of subdirectories
imageFiles = [[os.path.join(dataDir,classNames[i],x)
              for x in os.listdir(os.path.join(dataDir,classNames[i]))]
              for i in range(numClass)]                     # 각 클래스 별 파일 이름들
numEach = [len(imageFiles[i]) for i in range(numClass)]     # 각 클래스 별 파일 갯수
imageFilesList = []                                         # 모든 파일이름
imageClass = []                                             # 각각의 파일들에 대한 클래스

for i in range(numClass):
    imageFilesList.extend(imageFiles[i])
    imageClass.extend([i]*numEach[i])

numTotal = len(imageClass)                                            # 전체 파일 갯수
imageWidth, imageHeight = Image.open(imageFilesList[0]).size          # 각 영상의 사이즈(width, height)

print("There are",numTotal,"images in",numClass,"distinct categories")
print("Label names:",classNames)
print("Label counts:",numEach)
print("Image dimensions:",imageWidth,"x",imageHeight)

In [ ]:
# 전체 이미지 중 9개를 랜덤으로 골라 3x3으로 레이블과 함께 그리기
# -- 여러번 실행하며 이미지들을 살펴보세요 --

plt.subplots(3,3,figsize=(8,8))
for i,k in enumerate(np.random.randint(numTotal, size=9)):
    im = Image.open(imageFilesList[k])
    arr = np.array(im)
    plt.subplot(3,3,i+1)
    plt.xlabel(classNames[imageClass[k]])
    plt.imshow(arr,cmap='gray',vmin=0,vmax=255)
plt.tight_layout()
plt.show()

In [ ]:
# 이미지 리스트 살펴보기
imageFilesList[0:10]

## 2. VGG16을 이용한 분류 실습 (w/ ImageNet pre-trained weight)
![대체 텍스트](https://www.cs.toronto.edu/~frossard/post/vgg16/vgg16.png)

In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

### 2.1. [Quiz] 순서가 섞인 layer들을 VGG16 구성에 맞게 배치해보세요.

In [ ]:
inputs = Input(shape=(224, 224, 3,), name="VGGInput")

In [ ]:
x = Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu')(x)
x = Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu')(x)
x = MaxPool2D(padding='same')(x)

In [ ]:
x = Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu')(x)
x = Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu')(x)
x = Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu')(x)
x = MaxPool2D(padding='same')(x)

In [ ]:
x = Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu')(x)
x = Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu')(x)
x = Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu')(x)
x = MaxPool2D(padding='same')(x)

In [ ]:
x = Conv2D(filters=4096, kernel_size=(7,7), padding='valid', activation='relu')(x)
x = Flatten()(x)
x = Dense(4096, activation='relu')(x)
pred = Dense(1000, activation='softmax')(x)

In [ ]:
x = Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu')(x)
x = Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu')(x)
x = Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu')(x)
x = MaxPool2D(padding='same')(x)

In [ ]:
x = Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu')(inputs)
x = Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu')(x)
x = MaxPool2D(padding='same')(x)

In [ ]:
model = Model(inputs=inputs, outputs=pred)

In [ ]:
model.summary()

### 2.2. VGG16 모델 불러오기

In [ ]:
from tensorflow.keras.applications import vgg16

# VGG16 모델 불러오기
model = vgg16.VGG16()

# 모델의 모양을 보여준다.
model.summary()

In [ ]:
# Model 구성도 plot
from IPython.display import Image

tf.keras.utils.plot_model(model, to_file='vgg16.png', show_shapes=True, show_layer_names=True)
Image(filename='vgg16.png', width=300)

**VGG16**

`keras.applications.vgg16.VGG16(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)`

VGG16 model, with weights pre-trained on ImageNet.

This model can be built both with 'channels_first' data format (channels, height, width) or 'channels_last' data format (height, width, channels).

The default input size for this model is 224x224.

**Arguments**

* include_top: whether to include the 3 fully-connected layers at the top of the network.
* weights: one of None (random initialization) or 'imagenet' (pre-training on ImageNet).
* input_tensor: optional Keras tensor (i.e. output of layers.Input()) to use as image input for the model.
* input_shape: optional shape tuple, only to be specified if include_top is False (otherwise the input shape has to be (224, 224, 3) (with 'channels_last' data format) or (3, 224, 224) (with 'channels_first' data format). It should have exactly 3 inputs channels, and width and height should be no smaller than 32. E.g. (200, 200, 3) would be one valid value.
* pooling: Optional pooling mode for feature extraction when include_top is False.
* classes: optional number of classes to classify images into, only to be specified if include_top is True, and if no weights argument is specified.

**tf.keras에서 제공되는 모델들 참고:** https://www.tensorflow.org/api_docs/python/tf/keras/applications

In [ ]:
# VGG16 모델을 이용해 prediction 하는 함수
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from IPython.display import display # 이미지 출력 함수

def predict_vgg16(model, filename) :
    # 이미지 파일을 읽고 화면에 표시
    image = load_img(filename)
    display(image)

    # 모델 사이즈로 이미지 파일을 읽기
    image = load_img(filename, target_size=(224, 224))

    # 이미지 데이터를 numpy로 변환
    image = img_to_array(image)

    # vgg16.preprocess_input()을 호출하기 위해 차원을 조정
    # 보통 모델을 여러 이미지를 한번에 호출.
    # 맨 앞의 1 : 이미지 갯수가 1개라는 것.
    # 두번째 224 : 가로
    # 세번째 224 : 세로
    # 네번째 3 : R, G, B 3개
    image = image.reshape((1, 224, 224, 3))

    # VGG16 모델 호출을 위해 데이터 전처리.
    # -255 ~ 255 사이 값으로 정규화한다.
    # 그리고 RGB를 BGR순으로 바꾼다.
    image = vgg16.preprocess_input(image)


    # 이미지를 모델에 적용
    yhat = model.predict(image)

    # 모델 적용된 결과를 파싱
    label = vgg16.decode_predictions(yhat)

    # 가장 확률이 높은 결과를 획득
    label = label[0][0]

    # 라벨과 라벨을 예측한 확률을 출력
    print('%s (%.2f%%)' % (label[1], label[2]*100))

In [ ]:
files = imageFilesList[0:10]

In [ ]:
for file in files:
  predict_vgg16(model, file)

### 2.3. Dataset 나누기: Train / Validation / Test

In [ ]:
validFrac = 0.2   # Define the fraction of images to move to validation dataset
testFrac = 0.2    # Define the fraction of images to move to test dataset
validList = []
testList = []
trainList = []

for i in range(numTotal):
    rann = np.random.random() # Randomly reassign images
    if rann < validFrac:
        validList.append(i)
    elif rann < testFrac + validFrac:
        testList.append(i)
    else:
        trainList.append(i)

nTrain = len(trainList)  # Count the number in each set
nValid = len(validList)
nTest = len(testList)
print("Training images =",nTrain,"\nValidation =",nValid,"\nTesting =",nTest)

In [ ]:
!mkdir ./train
!mkdir ./valid
!mkdir ./test

In [ ]:
import shutil
from tqdm import tqdm

for i in tqdm(range(len(trainList))):
  root, clas, src = imageFilesList[trainList[i]].split('/')
  dest = os.path.join('./train',clas,src)
  if not os.path.exists(os.path.join('./train',clas)):
    os.mkdir(os.path.join('./train',clas))
  shutil.copy(imageFilesList[trainList[i]], dest)

for i in tqdm(range(len(validList))):
  root, clas, src = imageFilesList[validList[i]].split('/')
  dest = os.path.join('./valid',clas,src)
  if not os.path.exists(os.path.join('./valid',clas)):
    os.mkdir(os.path.join('./valid',clas))
  shutil.copy(imageFilesList[validList[i]], dest)


for i in tqdm(range(len(testList))):
  root, clas, src = imageFilesList[testList[i]].split('/')
  dest = os.path.join('./test',clas,src)
  if not os.path.exists(os.path.join('./test',clas)):
    os.mkdir(os.path.join('./test',clas))
  shutil.copy(imageFilesList[testList[i]], dest)

### 2.4. Image Data Generator 정의 (+Data Augmentation)
**Keras API - ImageDataGenerator: 일정한 규칙으로 만들어진 폴더구조에서 데이터셋을 자동으로 불러와 학습에 사용할 수 있게 도와주는 API**
![대체 텍스트](https://miro.medium.com/max/875/1*HpvpA9pBJXKxaPCl5tKnLg.jpeg)
https://medium.com/@vijayabhaskar96/tutorial-image-classification-with-keras-flow-from-directory-and-generators-95f75ebe5720  
**Data augmentation: 데이터에 다양한 형태의 변화를 임의로 생성하여 데이터의 갯수와 다양성을 증가시키는 방법**
![대체 텍스트](https://miro.medium.com/max/1250/1*rvwzKkvhlDN3Wo_4Oay_4Q.png)
https://medium.com/@thimblot/data-augmentation-boost-your-image-dataset-with-few-lines-of-python-155c2dc1baec


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Dropout

def preprocess_input_vgg(x):
    X = np.expand_dims(x, axis=0)
    X = preprocess_input(X)
    return X[0]


train_dir = './train'
validation_dir = './valid'
test_dir = './test'
batch_size = 32
image_size = 224

# 학습에 사용될 이미지 데이터 생성기
train_datagen = ImageDataGenerator(
      preprocessing_function=preprocess_input_vgg,
      rotation_range=20, # 회전 최대 20도
      width_shift_range=0.2, # 좌우 이동
      height_shift_range=0.2, # 상하 이동
      horizontal_flip=True, # 좌우 반전
      vertical_flip=True, # 상하 반전
      )

# 검증에 사용될 이미지 데이터 생성기
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg)

# 테스트에 사용될 이미지 데이터 생성기
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg)


# 학습에 사용될 데이터 생성기
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)

# 검증에 사용될 데이터 생성기
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

class_num=len(train_generator.class_indices)

custom_labels = list(validation_generator.class_indices.keys())

### 2.5. VGG16 as a Feature Extractor
![대체 텍스트](https://miro.medium.com/max/875/1*W91k18rRAZfJnsM8bhUDXA.png)
https://towardsdatascience.com/a-comprehensive-hands-on-guide-to-transfer-learning-with-real-world-applications-in-deep-learning-212bf3b2f27a

In [ ]:
# 모델 불러오기
vgg_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))
vgg_model.summary()

# Convolution Layer를 학습되지 않도록 고정
for layer in vgg_model.layers:
    layer.trainable = False

In [ ]:
# 새로운 모델 생성하기
last = vgg_model.output

# VGG16모델에 Fully Connected부분을 재구성해서 추가
x = Flatten()(last)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
pred = Dense(class_num, activation='softmax')(x)

model = Model(vgg_model.input, pred)

In [ ]:
# 새로운 모델 요약
model.summary()

In [ ]:
# 새로운 모델 저장
vgg16_model_path = 'vgg16_finetuning.h5'

model.save(vgg16_model_path)

### 2.6. 새로운 모델 학습

In [ ]:
import math
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import RMSprop

# 모델 로딩
model = load_model(vgg16_model_path)

# 모델 컴파일
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(learning_rate=1e-4),
              metrics=['acc'])

# 모델 학습
history = model.fit(
      x=train_generator,
      steps_per_epoch=100 ,
      epochs=2,
      validation_data=validation_generator,
      validation_steps=math.ceil(validation_generator.samples/validation_generator.batch_size),
      verbose=1)

# 모델 저장
model.save(vgg16_model_path)

### 2.7. 학습 결과 시각화

In [ ]:
acc = history.history['acc']
loss = history.history['loss']
valacc = history.history['val_acc']
valloss = history.history['val_loss']


epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='accuracy')
plt.plot(epochs, loss, 'r', label='loss')
plt.plot(epochs, valacc, 'b--', label='val_accuracy')
plt.plot(epochs, valloss, 'r--', label='val_loss')
plt.title('accuracy and loss')
plt.legend()

plt.show()

### 2.8. 학습된 모델을 이용해 Test 데이터에 대한 Prediction

In [ ]:
# 테스트에 사용될 데이터 생성기
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(image_size, image_size),
        batch_size=64,
        class_mode='categorical',
        shuffle=False)

In [ ]:
model.evaluate(test_generator, steps=math.ceil(test_generator.samples/test_generator.batch_size), verbose=1)

## 3. VGG16을 이용한 분류 실습 (from Scratch)

In [ ]:
# 모델 불러오기
vgg_model = vgg16.VGG16(weights=None, include_top=False, input_shape=(image_size, image_size, 3))
vgg_model.summary()

In [ ]:
last = vgg_model.output

In [ ]:
# VGG16모델에 Fully Connected부분을 재구성해서 추가
x = Flatten()(last)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
pred = Dense(class_num, activation='softmax')(x)

model = Model(vgg_model.input, pred)

In [ ]:
# 새로운 모델 요약
model.summary()

In [ ]:
# 새로운 모델 저장
vgg16_model_path = 'vgg16_scratch.h5'

model.save(vgg16_model_path)

In [ ]:
# 모델 로딩
model = load_model(vgg16_model_path)

# 모델 컴파일
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(learning_rate=1e-4),
              metrics=['acc'])

# 모델 학습
history = model.fit(
      train_generator,
      steps_per_epoch=100 ,
      epochs=2,
      validation_data=validation_generator,
      validation_steps=math.ceil(validation_generator.samples/validation_generator.batch_size),
      verbose=1)

# 모델 저장
model.save(vgg16_model_path)

In [ ]:
acc = history.history['acc']
loss = history.history['loss']
valacc = history.history['val_acc']
valloss = history.history['val_loss']


epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='accuracy')
plt.plot(epochs, loss, 'r', label='loss')
plt.plot(epochs, valacc, 'b--', label='val_accuracy')
plt.plot(epochs, valloss, 'r--', label='val_loss')
plt.title('accuracy and loss')
plt.legend()

plt.show()

In [ ]:
# For prediction purposes
y_pred = model.predict(test_generator, steps=math.ceil(test_generator.samples/test_generator.batch_size), verbose=1)
y_pred1 = np.argmax(y_pred, axis=1)

## 4. Confusion Matrix Visualization

In [ ]:
import pandas as pd
import seaborn as sn
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# y_test labeling
y_test = test_generator.labels

In [ ]:
# calculate confusion matrix for the predicted dataset
cm = confusion_matrix(y_test, y_pred1)

In [ ]:
# make a dataframe using cm array
df_cm = pd.DataFrame(cm, index = [i for i in classNames], columns = [i for i in classNames])

In [ ]:
# plot confusion matrix
plt.figure(figsize = (10, 7))
sn.heatmap(df_cm, annot=True)

In [ ]:
# classification report generation: precision, recall, f1-score.
print(classification_report(y_test, y_pred1, target_names=classNames))

## 5. VGG16 + Grad-CAM
![대체 텍스트](https://camo.githubusercontent.com/450498bd998fd99d51b647d2b6c8631e94585522/687474703a2f2f692e696d6775722e636f6d2f4a614762645a352e706e67)
**Grad-CAM: Why did you say that? Visual Explanations from Deep Networks via Gradient-based Localization**  
Ramprasaath R. Selvaraju, Abhishek Das, Ramakrishna Vedantam, Michael Cogswell, Devi Parikh, Dhruv Batra  
https://arxiv.org/abs/1610.02391  
  
**Example: 'Boxer'**  
![대체 텍스트](https://github.com/PowerOfCreation/keras-grad-cam/raw/master/examples/cat_dog.png)
![대체 텍스트](https://github.com/PowerOfCreation/keras-grad-cam/raw/master/examples/cat_dog_242_gradcam.jpg)
![대체 텍스트](https://github.com/PowerOfCreation/keras-grad-cam/raw/master/examples/cat_dog_242_guided_gradcam.jpg)

In [ ]:
inum = random.randrange(0,len(imageFilesList))
qimage0 = load_img(imageFilesList[inum], target_size=(224, 224))
qimage  = img_to_array(qimage0)
qimage  = qimage.reshape((1, 224, 224, 3))
qimage  = vgg16.preprocess_input(qimage)

In [ ]:
plt.imshow(qimage0)

In [ ]:
def grad_cam(input_model, image):
    nb_classes = 6  # 클래스 숫자
    preds = input_model.predict(image)
    predicted_label = np.argmax(preds[0])

    grad_model = tf.keras.Model(
        [input_model.input], [input_model.get_layer('block5_conv3').output, input_model.output]
    )

    with tf.GradientTape() as tape:
        image = tf.cast(image, tf.float32)
        conv_outputs, predictions = grad_model(image)

        if isinstance(predictions, list):
            predictions = predictions[0]

        loss = predictions[:, predicted_label]

    grad = tape.gradient(loss, conv_outputs)[0]
    weights = tf.reduce_sum(grad, axis=(0, 1))
    cam = tf.reduce_sum(tf.multiply(weights, conv_outputs[0]), axis=-1)
    cam = cv2.resize(cam.numpy(), (224, 224))
    cam = (cam - cam.min()) / (cam.max() - cam.min())
    cam = cv2.applyColorMap(
        cv2.cvtColor((cam * 255).astype("uint8"), cv2.COLOR_GRAY2BGR), cv2.COLORMAP_JET
    )

    return np.uint8(cam), predicted_label

In [ ]:
inum = random.randrange(0,len(imageFilesList))
qimage0 = load_img(imageFilesList[inum], target_size=(224, 224))
qimage  = img_to_array(qimage0)
qimage  = qimage.reshape((1, 224, 224, 3))
qimage  = vgg16.preprocess_input(qimage)

# CAM 출력
cam, plabel = grad_cam(model, qimage)
plt.title('T:'+classNames[imageClass[inum]]+', P:'+classNames[plabel])
plt.imshow(qimage0)
plt.imshow(cam, alpha=.5)